## Parâmetros
Vamos usar
+ $d \ge 6$ --
  parâmetro de segurança
+ $N$ --
  um primo maior ou igual  2^d
  
+ $p$ --
  um pequeno primo  3,  5 ou  7
 
+ $q$ --
  um primo da ordem de grandeza de  $p*N$
 

 

In [1]:
d = 6

N = next_prime(1 << d)
p = 3
q = next_prime(p*N)

print d, q, p, N

6 211 3 67


### anéis de polinómios

In [2]:
Z.<x>  = ZZ[]        # polinómios de coeficientes inteiros
Q.<x>  = PolynomialRing(GF(q),name='x').quotient(x^N-1)

### geração aleatória, arredondamento módulo $q$ e compração módulo $p$

In [2]:
def vec():
    return  [choice([-1,0,1]) for k in range(N)]

# arredondamento módulo 'q'
def qrnd(f):    # argumento em 'Q'
    qq = (q-1)//2 ; ll = map(lift,f.list())
    return [n if n <= qq else n - q  for n in ll]

# arredondamento módulo 'p'
def prnd(l):
    pp = (p-1)//2
    rr = lambda x: x if x <= pp else x - p        
    return [rr(n%p) if n>=0 else -rr((-n)%p) for n in l]

# comparação módulo 'p'
#def equalp(x,y,p):
#    return  all([(a-b)%p == 0  for (a,b) in zip(x,y)] )

### classe NTRU

In [1]:
class NTRU(object):
    def __init__(self):
        # calcular um 'f' invertível
        f = Q(0)
        while not f.is_unit():
            F = Q(vec()); f = 1 + p*F
        # gerar as chaves
        G = Q(vec()) ; g = p*G
        self.f = f
        self.h = f^(-1) * g
        
    def encrypt(self,m):
        r = Q(vec()) 
        return r*self.h + Q(m)

    def decrypt(self,e):
        a = e*self.f
        return prnd(qrnd(a))


### Teste

In [5]:
# Uma instância NTRU
K = NTRU() 
# Uma mensagem aleatória
m = vec(N)
# Cifrar
e = K.encrypt(m)  
# Decifrar e Verificar
m == K.decrypt(e)


True

NameError: name 'vec' is not defined

## Reticulado $L(h)$

In [6]:
# Construção da matriz geradora por blocos

import sage.modules.free_module_integer as fmi
# http://doc.sagemath.org/html/en/reference/modules/sage/modules/free_module_integer.html

class Lat(NTRU):
    def __init__(self):
        super(Lat,self).__init__()
        B1 = identity_matrix(ZZ,N); Bq = q*B1; B0 = matrix(ZZ,N,N,[0]*(N^2))
        h = qrnd(self.h)
        # rodar um vetor
        H = [h]
        for k in range(N-1):
            h = [h[-1]] + h[:-1]   # shift right rotate
            H = H + [h]
        H = matrix(ZZ,N,N,H)
        self.L = fmi.IntegerLattice(block_matrix([[Bq,B0],[H,B1]]))      

In [7]:
l = Lat()

In [8]:
l.L

Free module of degree 134 and rank 134 over Integer Ring
User basis matrix:
134 x 134 dense matrix over Integer Ring